In [ ]:
import numpy as np
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input, Cropping2D, Flatten
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
from tensorflow.keras.callbacks import ModelCheckpoint


# Cuda
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("GPU detected")
    except:
        print("GPU not detected")
        pass
else :
    print("GPU not detected") 

def unet_model(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)
    
    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Dropout(0.3)(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Decoder
    u4 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = Concatenate()([u4, c2])
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u4)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u5 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = Concatenate()([u5, c1])
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u5)
    c5 = Dropout(0.1)(c5)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    model = Model(inputs, outputs)
    return model

# Load images
def load_images(images_folder, masks_folder, target_size=(128, 128)):
    image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
    mask_files = [f for f in os.listdir(masks_folder) if os.path.isfile(os.path.join(masks_folder, f))]

    assert len(image_files) == len(mask_files), "Mismatach number of images and masks."

    images = []
    masks = []

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(images_folder, img_file)
        mask_path = os.path.join(masks_folder, mask_file)

        image = load_img(img_path, target_size=target_size)
        mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")

        # Normalize images
        image_array = img_to_array(image) / 255.0  
        mask_array = img_to_array(mask) / 255.0    

        images.append(image_array)
        masks.append(mask_array)

    return np.array(images), np.array(masks)

train_folder = '/kaggle/input/brain-tumor/train'
train_images_folder = os.path.join(train_folder, 'images')
train_masks_folder = os.path.join(train_folder, 'masks')

test_folder = '/kaggle/input/brain-tumor/test'
test_images_folder = os.path.join(test_folder, 'images')
test_masks_folder = os.path.join(test_folder, 'masks')

train_images, train_masks = load_images(train_images_folder, train_masks_folder)
test_images, test_masks = load_images(test_images_folder, test_masks_folder)

# Compile model
model_path = '/kaggle/input/last-data/last_model.keras'
if os.path.exists(model_path):
    print("Loading existing model...")
    model = tf.keras.models.load_model(model_path)
else:
    print("Creating new model...")
    model = unet_model(input_shape=(128, 128, 3))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
checkpoint_path = "/kaggle/working/best_model.keras"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_loss',  # Monitor validation loss
    save_best_only=True,  # Save only the best model based on validation loss
    verbose=1
)

# Train the model with the checkpoint callback
history = model.fit(
    train_images, train_masks, 
    validation_data=(test_images, test_masks),
    batch_size=32,
    epochs=20,
    verbose=1,
    callbacks=[checkpoint_callback]  # Add the checkpoint callback
)

GPU not detected
Loading existing model...
Epoch 1/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9912 - loss: 0.0245 
Epoch 1: val_loss improved from inf to 0.03493, saving model to /kaggle/working/best_model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 1129s 17s/step - accuracy: 0.9912 - loss: 0.0245 - val_accuracy: 0.9872 - val_loss: 0.0349
Epoch 2/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9911 - loss: 0.0248 
Epoch 2: val_loss improved from 0.03493 to 0.02426, saving model to /kaggle/working/best_model.keras
67/67 ━━━━━━━━━━━━━━━━━━━━ 1121s 17s/step - accuracy: 0.9911 - loss: 0.0247 - val_accuracy: 0.9914 - val_loss: 0.0243
Epoch 3/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9922 - loss: 0.0214 
Epoch 3: val_loss did not improve from 0.02426
67/67 ━━━━━━━━━━━━━━━━━━━━ 1145s 17s/step - accuracy: 0.9922 - loss: 0.0214 - val_accuracy: 0.9913 - val_loss: 0.0244
Epoch 4/20
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - accuracy: 0.9921 - loss: 0.0220 
Epoch 4: val_loss 

In [5]:
from sklearn.metrics import confusion_matrix
import numpy as np
from tensorflow.keras.layers import Activation, Dense, Dropout, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Input, Cropping2D, Flatten
from keras.models import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

def load_images(images_folder, masks_folder, target_size=(128, 128)):
    image_files = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f))]
    mask_files = [f for f in os.listdir(masks_folder) if os.path.isfile(os.path.join(masks_folder, f))]

    assert len(image_files) == len(mask_files), "Mismatach number of images and masks."

    images = []
    masks = []

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(images_folder, img_file)
        mask_path = os.path.join(masks_folder, mask_file)

        image = load_img(img_path, target_size=target_size)
        mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")

        # Normalize images
        image_array = img_to_array(image) / 255.0  
        mask_array = img_to_array(mask) / 255.0    

        images.append(image_array)
        masks.append(mask_array)

    return np.array(images), np.array(masks)


test_folder = '/kaggle/input/brain-tumor/test'
test_images_folder = os.path.join(test_folder, 'images')
test_masks_folder = os.path.join(test_folder, 'masks')
test_images, test_masks = load_images(test_images_folder, test_masks_folder)

# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = (y_pred.flatten() > 0.5).astype(int)

    # Confusion matrix elements
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    # Pixel Accuracy
    pixel_accuracy = (tp + tn) / (tp + tn + fp + fn)

    # Jaccard's Index (IoU)
    iou = tp / (tp + fn + fp)

    # Dice Coefficient
    dice = (2 * tp) / (2 * tp + fn + fp)

    return pixel_accuracy, iou, dice

# Load the best model
model = tf.keras.models.load_model('/kaggle/input/model1/best_model (1).keras')


test_predictions = model.predict(test_images)
pixel_acc, mean_iou, dice_coeff = calculate_metrics(test_masks, test_predictions)

print(f"Pixel Accuracy: {pixel_acc:.4f}")
print(f"Mean IoU (Jaccard's Index): {mean_iou:.4f}")
print(f"Mean Dice Coefficient: {dice_coeff:.4f}")

29/29 ━━━━━━━━━━━━━━━━━━━━ 124s 4s/step
Pixel Accuracy: 0.9845
Mean IoU (Jaccard's Index): 0.1514
Mean Dice Coefficient: 0.2630
